In [2]:
import json
from collections import Counter

from SPARQLWrapper import SPARQLWrapper, JSON
import time

from tqdm import tqdm

import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np

import faiss

/home/jovyan/chepurova/Text2KG/venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Collecting relation names and constraints

In [2]:
ONTOLOGY_MAPPINGS_DIR = "../utils/ontology_mappings/"

In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON

PROP_2_LABEL = {}
PROP_2_DATA_TYPE = {}

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# SPARQL query for properties with data types: Item, Quantity, Point in time
query = """
SELECT ?property ?propertyLabel ?typeLabel WHERE {
  ?property a wikibase:Property .
  ?property wikibase:propertyType ?type .
  
  VALUES ?type { wikibase:WikibaseItem wikibase:Quantity wikibase:Time }
  
  BIND(
    IF(?type = wikibase:WikibaseItem, "Item",
      IF(?type = wikibase:Quantity, "Quantity",
        IF(?type = wikibase:Time, "Point in time", "Unknown")
      )
    ) AS ?typeLabel
  )
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)

try:
    results = sparql.query().convert()

    # Extract and print property IDs with their labels and data types
    for result in results["results"]["bindings"]:
        prop = result["property"]["value"].split("/")[-1]
        label = result.get("propertyLabel", {}).get("value", "No label")
        data_type = result.get("typeLabel", {}).get("value", "Unknown")

        PROP_2_LABEL[prop] = label
        PROP_2_DATA_TYPE[prop] = data_type        

except Exception as e:
    print(f"Error executing SPARQL query: {e}")

In [4]:
len(PROP_2_LABEL), len(PROP_2_DATA_TYPE)

(2410, 2410)

In [5]:
with open(ONTOLOGY_MAPPINGS_DIR+"prop2data_type.json", 'w') as f:
    json.dump(PROP_2_DATA_TYPE, f)

In [6]:
with open(ONTOLOGY_MAPPINGS_DIR+"prop2label.json", 'w') as f:
    json.dump(PROP_2_LABEL, f)

In [7]:
with open(ONTOLOGY_MAPPINGS_DIR+'prop2constraints.json', 'r') as f:
    constraint_dict = json.load(f)

In [8]:
constraint_dict = {key: value for key, value in constraint_dict.items() if key in PROP_2_LABEL}
len(constraint_dict)

2317

In [9]:
data_constraints = ["subject type constraint", "value-type constraint" ]
                            # "allowed-entity-types constraint", 
                            #   "one-of constraint"]

In [10]:
prop2constraints = {}

for prop in constraint_dict:
    
    tmp = {}
    
    for constraint in data_constraints:
        if constraint in constraint_dict[prop]:
            # P2308 - class
            if 'P2308' in constraint_dict[prop][constraint]:
                tmp[constraint] = constraint_dict[prop][constraint]['P2308']
        
    prop2constraints[prop] = tmp.copy()

In [11]:
prop2constraints['P2058']

{'subject type constraint': ['Q131569'],
 'value-type constraint': ['Q4164871', 'Q43229']}

In [12]:
len(prop2constraints)

2317

In [13]:
properties = [prop for prop in prop2constraints.keys()]
len(set(properties))

2317

In [14]:
properties

['P1661',
 'P9897',
 'P3156',
 'P9086',
 'P2629',
 'P8476',
 'P8477',
 'P6452',
 'P9028',
 'P853',
 'P6657',
 'P7603',
 'P5475',
 'P2758',
 'P3402',
 'P880',
 'P9887',
 'P2643',
 'P3501',
 'P3216',
 'P9798',
 'P9126',
 'P6731',
 'P10273',
 'P10300',
 'P8328',
 'P1032',
 'P660',
 'P4105',
 'P2756',
 'P8901',
 'P852',
 'P9279',
 'P470',
 'P1087',
 'P2021',
 'P3823',
 'P6438',
 'P2371',
 'P4437',
 'P2150',
 'P1981',
 'P1731',
 'P2747',
 'P6697',
 'P5070',
 'P4010',
 'P5040',
 'P5041',
 'P5042',
 'P1033',
 'P2560',
 'P9866',
 'P916',
 'P1414',
 'P9325',
 'P1125',
 'P2597',
 'P7295',
 'P5425',
 'P5537',
 'P11547',
 'P8874',
 'P1081',
 'P3306',
 'P4628',
 'P7573',
 'P5150',
 'P579',
 'P5201',
 'P3428',
 'P6069',
 'P6095',
 'P6789',
 'P141',
 'P814',
 'P11593',
 'P2127',
 'P5900',
 'P5386',
 'P3650',
 'P5152',
 'P10994',
 'P3096',
 'P3818',
 'P2684',
 'P2564',
 'P1145',
 'P8026',
 'P2051',
 'P1657',
 'P8889',
 'P2377',
 'P5970',
 'P2784',
 'P9895',
 'P1088',
 'P1611',
 'P2363',
 'P9888',
 'P3

In [15]:
with open(ONTOLOGY_MAPPINGS_DIR+'subject_object_constraints.json', 'w') as f:
    json.dump(prop2constraints, f)

In [16]:
with open(ONTOLOGY_MAPPINGS_DIR+'subject_object_constraints.json', 'r') as f:
    prop2constraint = json.load(f)

In [17]:
subj2prop_constraint = {}
obj2prop_constraint = {}

for prop, constraints in prop2constraint.items():

    if 'subject type constraint' in constraints:
        for constraint_entity in constraints['subject type constraint']:
            if constraint_entity in subj2prop_constraint:
                subj2prop_constraint[constraint_entity].append(prop)
            else:
                subj2prop_constraint[constraint_entity] = [prop]
    
    if 'value-type constraint' in constraints: 
        for constraint_entity in constraints['value-type constraint']:
            if constraint_entity in obj2prop_constraint:
                obj2prop_constraint[constraint_entity].append(prop)
            else:
                obj2prop_constraint[constraint_entity] = [prop]


obj2prop_constraint

{'Q110910264': ['P9897'],
 'Q26708074': ['P3156'],
 'Q23790218': ['P2629'],
 'Q23683472': ['P853'],
 'Q17300291': ['P5475', 'P5425', 'P5523', 'P5753'],
 'Q5300': ['P880'],
 'Q1466064': ['P880'],
 'Q80831': ['P880', 'P7501'],
 'Q610398': ['P880'],
 'Q122967152': ['P880'],
 'Q10335893': ['P9887'],
 'Q4223026': ['P2643'],
 'Q13024547': ['P3501'],
 'Q26678730': ['P3216'],
 'Q43229': ['P9126',
  'P8413',
  'P1817',
  'P1416',
  'P1877',
  'P11812',
  'P945',
  'P1001',
  'P748',
  'P485',
  'P9493',
  'P50',
  'P11811',
  'P3320',
  'P3301',
  'P562',
  'P195',
  'P6241',
  'P88',
  'P767',
  'P3931',
  'P736',
  'P170',
  'P8786',
  'P1591',
  'P2058',
  'P1343',
  'P287',
  'P178',
  'P8791',
  'P61',
  'P10527',
  'P1028',
  'P98',
  'P5769',
  'P108',
  'P8001',
  'P10661',
  'P8571',
  'P112',
  'P355',
  'P110',
  'P7010',
  'P10836',
  'P1840',
  'P2378',
  'P157',
  'P7514',
  'P1018',
  'P276',
  'P4647',
  'P193',
  'P126',
  'P176',
  'P463',
  'P7888',
  'P4353',
  'P800',
  'P4

In [18]:
with open(ONTOLOGY_MAPPINGS_DIR+'obj2prop_constraint.json', 'w') as f:
    json.dump(obj2prop_constraint, f)

with open(ONTOLOGY_MAPPINGS_DIR+'subj2prop_constraint.json', 'w') as f:
    json.dump(subj2prop_constraint, f)

In [19]:
# c = Counter()

# for prop in value_rel_dict:

#     for key in value_rel_dict[prop]:
#         for item in value_rel_dict[prop][key]["P2309"]:
#             c[item] += 1

# # Q21503252 - instance of 
# # Q30208840 - instance or subclass of 
# # Q21514624 - subclass of

## Colecting entities' metadata

In [20]:
entities = set()
for prop in prop2constraints:

    for const_type in prop2constraints[prop]:
        for entity in prop2constraints[prop][const_type]:
            entities.add(entity)
entities = list(entities)

In [21]:
len(entities)

3138

### Collecting entities' hierarchy of superclasses

In [22]:
from tenacity import retry, wait_random_exponential, before_sleep_log

@retry(wait=wait_random_exponential(multiplier=1, max=60))
def get_subclass_hierarchy(entity_id):
        # Wikidata SPARQL endpoint
      sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

      # SPARQL query to get all subclasses (direct and indirect) of the given entity
      query = f"""
      SELECT DISTINCT ?subclass ?subclassLabel WHERE {{
          {{
              wd:{entity_id} wdt:P31/wdt:P279* ?subclass.
          }}
            UNION
          {{
              wd:{entity_id} wdt:P279* ?subclass.
          }}
      }}
      """
    # SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}

      sparql.setQuery(query)
      sparql.setReturnFormat(JSON)

      # Execute the query and parse the results
      results = sparql.query().convert()

      subclass_hierarchy = []

      # Extract subclass information from the query results
      for result in results["results"]["bindings"]:
          subclass_id = result["subclass"]["value"].split("/")[-1]
          subclass_hierarchy.append(subclass_id)

      return subclass_hierarchy

ENTITY_2_HIERARCHY = {}
for entity_id in tqdm(entities):
    hierarchy = get_subclass_hierarchy(entity_id)
    ENTITY_2_HIERARCHY[entity_id] = hierarchy

len(ENTITY_2_HIERARCHY)

  0%|          | 0/3138 [00:00<?, ?it/s]

100%|██████████| 3138/3138 [22:46<00:00,  2.30it/s]  


3138

In [23]:
ents = []
for item in ENTITY_2_HIERARCHY.values():
    ents.extend(item)
len(set(ents)), len(entities)

(6731, 3138)

In [24]:
for entity in tqdm(ENTITY_2_HIERARCHY):
    filtered_super_entities = [item for item in ENTITY_2_HIERARCHY[entity] if item in entities]
    super_entities = ENTITY_2_HIERARCHY[entity]
    ENTITY_2_HIERARCHY[entity] = filtered_super_entities

100%|██████████| 3138/3138 [00:02<00:00, 1340.44it/s]


In [25]:
ents = []
for item in ENTITY_2_HIERARCHY.values():
    ents.extend(item)
len(set(ents)), len(entities)

(3138, 3138)

In [27]:
with open(ONTOLOGY_MAPPINGS_DIR + 'entity_hierarchy.json', 'w') as f:
    json.dump(ENTITY_2_HIERARCHY, f)

### Collecting entities' labels

In [28]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

BATCH_SIZE = 50

def fetch_labels(batch):
    entity_values = " ".join(f"wd:{entity}" for entity in batch)
    
    query = f"""
    SELECT ?entity ?entityLabel WHERE {{
      VALUES ?entity {{ {entity_values} }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        results = sparql.query().convert()
        return {
            result["entity"]["value"].split("/")[-1]: result.get("entityLabel", {}).get("value", "No label")
            for result in results["results"]["bindings"]
        }
    except Exception as e:
        print(f"Error with batch {batch[:5]}...: {e}")
        return {}

ENTITY_2_LABEL = {}

# Process in batches
for i in range(0, len(entities), BATCH_SIZE):
    batch = entities[i:i + BATCH_SIZE]
    print(f"Processing batch {i // BATCH_SIZE + 1}/{(len(entities) // BATCH_SIZE) + 1}")
    
    labels = fetch_labels(batch)
    ENTITY_2_LABEL.update(labels)
    
    time.sleep(2)

# for entity, label in all_labels.items():
#     print(f"{entity}: {label}")

Processing batch 1/63
Processing batch 2/63
Processing batch 3/63
Processing batch 4/63
Processing batch 5/63
Processing batch 6/63
Processing batch 7/63
Processing batch 8/63
Processing batch 9/63
Processing batch 10/63
Processing batch 11/63
Processing batch 12/63
Processing batch 13/63
Processing batch 14/63
Processing batch 15/63
Processing batch 16/63
Processing batch 17/63
Processing batch 18/63
Processing batch 19/63
Processing batch 20/63
Processing batch 21/63
Processing batch 22/63
Processing batch 23/63
Processing batch 24/63
Processing batch 25/63
Processing batch 26/63
Processing batch 27/63
Processing batch 28/63
Processing batch 29/63
Processing batch 30/63
Processing batch 31/63
Processing batch 32/63
Processing batch 33/63
Processing batch 34/63
Processing batch 35/63
Processing batch 36/63
Processing batch 37/63
Processing batch 38/63
Processing batch 39/63
Processing batch 40/63
Processing batch 41/63
Processing batch 42/63
Processing batch 43/63
Processing batch 44/

In [29]:
len(ENTITY_2_LABEL)

3138

In [31]:
base_ontology_triplets = []
for prop, constraint in prop2constraints.items():
    triple = {}
    verb_prop = PROP_2_LABEL[prop]
    triple["property"] = verb_prop
    for constraint_type in constraint:
        if constraint_type == "subject type constraint":
            triple["subject"] = []
            subjects = [ENTITY_2_LABEL[entity] for entity in constraint[constraint_type]]
            triple["subject"].extend(subjects)
        
        if constraint_type == "value-type constraint":
            triple["value"] = []
            values = [ENTITY_2_LABEL[entity] for entity in constraint[constraint_type]]
            triple["value"].extend(values)
    base_ontology_triplets.append(triple)

In [32]:
base_ontology_triplets

[{'property': 'Alexa rank', 'subject': ['website']},
 {'property': 'App Store age rating',
  'subject': ['video game', 'mobile app'],
  'value': ['Apple App Store rating category']},
 {'property': 'Australian Classification',
  'subject': ['film',
   'television program',
   'video game',
   'video game compilation',
   'television series episode',
   'trailer'],
  'value': ['Australian Classification category']},
 {'property': 'BAMID film rating', 'subject': ['film']},
 {'property': 'BBFC rating',
  'subject': ['film',
   'television program',
   'video game',
   'television series episode'],
  'value': ['BBFC classification category']},
 {'property': 'BTI Governance Index', 'subject': ['country']},
 {'property': 'BTI Status Index', 'subject': ['country']},
 {'property': 'CBFC rating', 'subject': ['film']},
 {'property': 'CCC classification', 'subject': ['video game']},
 {'property': 'CERO rating',
  'subject': ['video game',
   'expansion pack',
   'video game compilation',
   'ficti

In [33]:
with open('ontology_entity2label.json', 'w') as f:
    json.dump(ENTITY_2_LABEL, f)

## Index utils

In [34]:
tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
model = AutoModel.from_pretrained('facebook/contriever').to('cuda:4')

In [35]:
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

def embed_entity_batch(entity_list):
    inputs = tokenizer(entity_list, padding=True, truncation=True, return_tensors='pt')

    outputs = model(**inputs.to('cuda:4'))
    embeddings = mean_pooling(outputs[0], inputs['attention_mask'])
    return embeddings

## Relation index

In [8]:
prop_items = list(PROP_2_LABEL.items())
prop_ids = [int(item[0][1:]) for item in prop_items]
prop_names = [item[1] for item in prop_items]
len(set(prop_ids)), len(set(prop_names))

(2408, 2408)

In [10]:
prop_embeddings = []
batch_size = 100

for i in tqdm(range(0, len(prop_names), batch_size)):

    if i + batch_size > len(prop_names):
        prop_list = prop_names[i: len(prop_names)]
    else:
        prop_list = prop_names[i: i + batch_size]

    prop_embeddings.append(embed_entity_batch(prop_list).detach().to('cpu'))

100%|██████████| 25/25 [00:01<00:00, 17.35it/s]


In [15]:
prop_output = np.array(torch.concat(prop_embeddings))
prop_output.shape

(2408, 768)

In [13]:
assert all(isinstance(id_, int) for id_ in prop_ids)
prop_ids[:5]

[6, 16, 17, 19, 20]

In [16]:
dim = prop_output.shape[1]
metric = faiss.METRIC_INNER_PRODUCT
prop_index = faiss.index_factory(dim, "IDMap,Flat", metric)

prop_index.add_with_ids(prop_output, prop_ids)

In [18]:
print(prop_index.is_trained)

True


In [20]:
from time import time

before = time()
distances, indices = prop_index.search(prop_output[100:103, :], 3)
after = time()
after - before

0.005139827728271484

In [21]:
indices, distances

(array([[  196,   690,   744],
        [  197,  3032,    81],
        [  199,   749, 12526]]),
 array([[1.4933411 , 1.1615491 , 1.0979967 ],
        [1.8069766 , 1.3875215 , 1.0804527 ],
        [1.7670994 , 0.9554285 , 0.93438303]], dtype=float32))

In [24]:
print([PROP_2_LABEL["P"+str(i)] for i in indices[0]])
print([PROP_2_LABEL["P"+str(i)] for i in indices[1]])
print([PROP_2_LABEL["P"+str(i)] for i in indices[2]])

['minor planet group', 'space group', 'asteroid family']
['adjacent station', 'adjacent building', 'connecting line']
['organizational divisions', 'parent organization', 'performing organization']


In [27]:
faiss.write_index(prop_index, "wikidata_relations.index")

## Indexing ontology labels

In [3]:
with open('ontology_entity2label.json', 'r') as f:
    ENTITY_2_LABEL = json.load(f)
ENTITY_2_LABEL

{'Q31855': 'research institute',
 'Q192276': 'measure',
 'Q830450': 'microform',
 'Q15331236': 'fictional family',
 'Q55833': 'supercontinent',
 'Q57812611': 'captive mammal',
 'Q5300': 'central processing unit',
 'Q65938634': 'quality control method',
 'Q49459835': 'fraternal hall',
 'Q103817': 'indigenous people',
 'Q188784': 'superhero',
 'Q24336466': 'mythical event',
 'Q3707858': 'type',
 'Q2989352': 'free software community',
 'Q13582682': 'classification',
 'Q43812': 'passport',
 'Q2806437': 'transport service',
 'Q55640599': 'group of chemical entities',
 'Q11415657': 'professional name',
 'Q1324545': 'blimp',
 'Q15132612': 'unit prefix',
 'Q102109876': 'ELSPA classification category',
 'Q105390172': 'Roman Catholic metropolitan archdiocese',
 'Q1278335': 'instrumentalist',
 'Q9096832': 'paleontological site',
 'Q216926': 'raga',
 'Q6056746': 'campaign',
 'Q51077473': 'Wikidata property related to time and duration',
 'Q20643955': 'human biblical figure',
 'Q6576792': 'online c

In [4]:
entity_items = list(ENTITY_2_LABEL.items())
entity_ids = [int(item[0][1:]) for item in entity_items]
entity_names = [item[1] for item in entity_items]
len(set(entity_ids)), len(set(entity_names))

(3138, 3088)

In [5]:
entity_embeddings = []
batch_size = 100

for i in tqdm(range(0, len(entity_names), batch_size)):

    if i + batch_size > len(entity_names):
        entity_list = entity_names[i: len(entity_names)]
    else:
        entity_list = entity_names[i: i + batch_size]

    entity_embeddings.append(embed_entity_batch(entity_list).detach().to('cpu'))

  0%|          | 0/32 [00:00<?, ?it/s]


NameError: name 'embed_entity_batch' is not defined

In [8]:
entity_output = np.array(torch.concat(entity_embeddings))
entity_output.shape

(3138, 768)

In [10]:
dim = entity_output.shape[1]
metric = faiss.METRIC_INNER_PRODUCT
entity_index = faiss.index_factory(dim, "IDMap,Flat", metric)

entity_index.add_with_ids(entity_output, entity_ids)

In [11]:
print(entity_index.is_trained)

True


In [12]:
from time import time

before = time()
distances, indices = entity_index.search(entity_output[100:103, :], 3)
after = time()
after - before

0.004317283630371094

In [13]:
indices, distances

(array([[ 1924249, 42014143,   849203],
        [  188968, 12558574,  2075301],
        [ 1522115,  1753139,  1146001]]),
 array([[2.6669273, 1.9941075, 0.8888495],
        [2.1201134, 1.3751981, 1.3191648],
        [1.8139822, 1.4395627, 1.2694929]], dtype=float32))

In [14]:
indices, distances

(array([[ 1924249, 42014143,   849203],
        [  188968, 12558574,  2075301],
        [ 1522115,  1753139,  1146001]]),
 array([[2.6669273, 1.9941075, 0.8888495],
        [2.1201134, 1.3751981, 1.3191648],
        [1.8139822, 1.4395627, 1.2694929]], dtype=float32))

In [15]:
print([ENTITY_2_LABEL["Q"+str(i)] for i in indices[0]])
print([ENTITY_2_LABEL["Q"+str(i)] for i in indices[1]])
print([ENTITY_2_LABEL["Q"+str(i)] for i in indices[2]])

['measurand', 'biomedical measurand type', 'electronic countermeasure']
['perspective', 'point of view', 'view']
['energy source', 'electric power source', 'light source']


In [38]:
distances, idx = entity_index.search(embed_entity_batch(['organization']).detach().cpu().numpy(), 10)
distances, idx

(array([[1.4062808, 1.199734 , 1.170858 , 1.1634574, 1.1296779, 1.1264762,
         1.1156702, 1.1106048, 1.0937347, 1.0656824]], dtype=float32),
 array([[   43229,  2659904,  5341295,  7210356,  4438121, 17149090,
          1530022,  4120211, 16519632,   163740]]))

In [48]:
print([ENTITY_2_LABEL["Q"+str(i)] for i in idx[0]])

['organization', 'government organization', 'educational organization', 'political organization', 'sports organization', 'armed organization', 'religious organization', 'regional organization', 'scientific organization', 'nonprofit organization']


In [16]:
faiss.write_index(entity_index, "wikidata_ontology_entities.index")

In [67]:
with open('prop2label.json', 'r') as f:
    PROP_2_LABEL = json.load(f)
# PROP_2_LABEL

In [54]:
[PROP_2_LABEL[prop] for prop in prop2obj_constraint["Q"+'43229']]

['Commons media contributed by',
 'academic appointment',
 'addressee',
 'affiliation',
 'after a work by',
 'afterward owned by',
 'allegiance',
 'applies to jurisdiction',
 'appointed by',
 'archives at',
 'artist files at',
 'author',
 'beforehand owned by',
 'board member',
 'broadcast by',
 'central bank/issuer',
 'collection',
 'collection creator',
 'commissioned by',
 'contributor to the creative work or subject',
 'copyright holder',
 'cover art by',
 'creator',
 'dedicated heritage entity',
 'defendant',
 'depositary',
 'described by source',
 'designed by',
 'developer',
 'digitised by',
 'discoverer or inventor',
 'documentation files at',
 'donated by',
 'editor',
 'editor-in-chief',
 'employer',
 'endorsed by',
 'exhibited creator',
 'external auditor',
 'founded by',
 'has subsidiary',
 'illustrator',
 'imprimatur granted by',
 'inker',
 'investigated by',
 'issued by',
 'killed by',
 'landscape architect',
 'language regulatory body',
 'location',
 'location of first pe

In [56]:
subj_prop_lens = []
obj_prop_lens = []

for props in prop2obj_constraint.values():
    obj_prop_lens.append(len(props))

for props in prop2subj_constraint.values():
    subj_prop_lens.append(len(props))

In [65]:
max(obj_prop_lens), np.mean(obj_prop_lens), np.median(obj_prop_lens)

(709, 2.4506237006237006, 1.0)

In [66]:
max(subj_prop_lens), np.mean(subj_prop_lens), np.median(subj_prop_lens)

(1039, 3.6275586620069897, 1.0)

In [7]:
# Example usage:
entity_id = "Q5"  # Human
hierarchy = get_subclass_hierarchy(entity_id)
for item in hierarchy:
    print(item)

object
person
individual
heterotroph
natural person
agent
vertebrate
person or organization
individual organism
anatomical entity
consumer
omnivore
individual animal
mammal
physical object
legal person
concrete object
organism
entity
animal
material entity
independent continuant
physical anatomical entity
continuant
class
class
group or class of organisms
group or class of physical objects
collective entity
abstract entity
organisms known by a particular common name
Q488383
Q215627
Q795052
Q159344
Q154954
Q24229398
Q12898224
Q106559804
Q10855152
Q27043950
Q72638
Q164509
Q26401003
Q110551885
Q223557
Q3778211
Q4406616
Q7239
Q35120
Q729
Q53617407
Q53617489
Q66394244
Q103940464
Q5127848
Q16889133
Q21871294
Q98119401
Q99527517
Q7048977
Q55983715
